In [130]:
import pandas as pd

In [131]:
def preProcessData(stockName, MA_ST, MA_LT):
    df = pd.read_csv("IntradayData_2018/"+stockName+".txt")
    df.columns = ['stock', 'date','time','openingPrice','high','low','closingPrice','d','e']
    df['MA_ST'] = df['openingPrice'].rolling(window=MA_ST).mean()
    df['MA_LT'] = df['openingPrice'].rolling(window=MA_LT).mean()
    
    dateList = list(set(df.date.values))
    dateList = sorted(dateList, key=abs, reverse=False)
    return df, dateList

In [161]:
def convertTime(timeString):
    timeArr = timeString.split(':')
    time = int(timeArr[0])*60 + int(timeArr[1])
    return time

def revConvertTime(timeInt):
    timeHr = timeInt/60
    timeMin = timeInt%60
    return str(timeHr) + ':' + str(timeMin)

def sellIt(buyPrice, sellPrice, timeBought, timeSold, verbose):
    profit = 100*(sellPrice - buyPrice*(1.0003535))/(1.0*buyPrice )
    if(verbose): 
        print(buyPrice, sellPrice, revConvertTime(timeBought), revConvertTime(timeSold), profit)
    return profit

def buyIt(sellPrice, buyPrice, timeBought, timeSold, verbose):
    profit = 100*(sellPrice - buyPrice*(1.0003535))/(1.0*buyPrice )
    if(verbose): 
        print(buyPrice, sellPrice, revConvertTime(timeBought), revConvertTime(timeSold), profit)
    return profit

def parseRow(row):
    time = convertTime(row[1].time)
    openingPrice = row[1].openingPrice
    low = row[1].low
    high = row[1].high
    closingPrice = row[1].closingPrice
    listPrice = 0.5*(openingPrice + closingPrice)
    diff = row[1].MA_LT - row[1].MA_ST
    return time, openingPrice, low, high, closingPrice, listPrice, diff

In [162]:
def dayScript(df, targetPercentage, stopLossPercentage, entryDifference, verbose = False):
    eomDays = 0
    boughtFlag = 0
    dailyTrades = 0
    totalProfit = 0
    buyPrice = 0
    sellPrice = 0
    
    sellMargin = 1 + targetPercentage
    buyMargin = 1 - targetPercentage

    stopLoss = 1 - stopLossPercentage
    stopGain = 1 + stopLossPercentage

    for row in df.iterrows():
        time, openingPrice, low, high, closingPrice, listPrice, diff = parseRow(row)       
        if(time >= 570 and time <900):
            if(boughtFlag == 0):
                if(diff>0 and diff<entryDifference*listPrice):
                    buyPrice = listPrice
                    sellPrice = 0
                    timeBought = time
                    timeSold = 0
                    boughtFlag = 1
                elif(diff<0 and diff>-1*entryDifference*listPrice):
                    sellPrice = listPrice
                    buyPrice = 0
                    timeSold = time
                    timeBought = 0
                    boughtFlag = -1
            elif(boughtFlag==1):
                if(high > buyPrice*sellMargin):
                    totalProfit+= sellIt(buyPrice, buyPrice*sellMargin, timeBought, time, verbose)
                    dailyTrades+=1
                    boughtFlag = 0
                    buyPrice = 0
                elif(buyPrice*stopLoss > low):
                    totalProfit+= sellIt(buyPrice, buyPrice*stopLoss, timeBought, time, verbose)
                    dailyTrades+=1
                    boughtFlag = 0
                    buyPrice = 0
            else:
                if(sellPrice*buyMargin > low):
                    totalProfit+= buyIt(sellPrice, sellPrice*buyMargin, timeSold, time, verbose)
                    dailyTrades+=1
                    boughtFlag = 0
                    sellPrice = 0
                elif(high > sellPrice*stopGain):
                    totalProfit+= buyIt(sellPrice, sellPrice*stopGain, timeSold, time, verbose)
                    dailyTrades+=1
                    boughtFlag = 0
                    sellPrice = 0
        elif(time >= 900):
            if(boughtFlag == 1):
                totalProfit+=sellIt(buyPrice, closingPrice, timeBought, time, verbose)
                dailyTrades+=1
                boughtFlag = 0
                buyPrice = 0
                eomDays += 1
            elif(boughtFlag == -1):
                totalProfit+=buyIt(sellPrice, closingPrice, timeSold, time, verbose)
                dailyTrades+=1
                boughtFlag = 0
                buyPrice = 0
                eomDays += 1

#     print('Total Profit : ',totalProfit)
    return totalProfit, dailyTrades, eomDays

In [163]:
df, dateList = preProcessData('RELIANCE',20,200)
# [1,2,4,8,12,16,20,30,40]
# [80,90,100,150,200,300,400,500]
profitArr = []
targetPercentage = 0.0015
#0.0005 - 0.0020
stopLossPercentage = 0.005
#0.002-0.010
entryDifference = 0.0001
#0.00001-0.0002
for date in dateList:
    new_df = df[(df.date == date)]
    profit, dailyTrades, eomDays = dayScript(new_df, targetPercentage, stopLossPercentage, entryDifference, verbose = False)
    print(date, profit, dailyTrades)
    profitArr.append(profit)
    
print('Eom days - ',eomDays)
print('Total Profit %= ',sum(profitArr))

(20180101, -0.5353499999999849, 1)
(20180102, 0.22952533800704045, 2)
(20180103, 0.11487533800701467, 1)
(20180104, 0.0888402527206717, 4)
(20180105, -0.532862437810926, 1)
(20180108, 0.092996254298751, 2)
(20180109, -0.3033370998038836, 3)
(20180110, 0.22930000000002643, 2)
(20180111, 0.22952533800702885, 2)
(20180112, 0.22930000000003598, 2)
(20180115, 0.11487533800701524, 1)
(20180116, 0, 0)
(20180117, 0.22952533800702515, 2)
(20180118, -0.4206999999999692, 2)
(20180119, 0.2293000000000291, 2)
(20180122, -0.5965191799622311, 2)
(20180123, 0.11487533800701184, 1)
(20180124, -0.476902405298616, 1)
(20180125, 0.11465000000002105, 1)
(20180129, -0.4206999999999685, 2)
(20180130, 0, 0)
(20180131, 0.229300000000037, 2)
(20180201, 0, 0)
(20180202, 0, 0)
(20180205, 0.22952533800703795, 2)
(20180206, 0.11465000000001124, 1)
(20180207, -0.42047466199298333, 2)
(20180208, 0.1146500000000159, 1)
(20180209, 0.22930000000003503, 2)
(20180212, -0.5328624378109209, 1)
(20180214, 0, 0)
(20180215, 0.